In [102]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split

from patsy import dmatrices
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [2]:
df = pd.read_csv("final_olympic_data.csv")

In [3]:
summer_df = df[df["season"] == "Summer"]
summer_df

,id,sex,age,height,weight,noc,year,season,city,sport,event_category,event,medal,region
0,2,M,23.0,170.0,60.0,CHN,2012,Summer,London,Judo,judo_half_lightweight,Extra-Lightweight,NaN,China
25,13,F,30.0,159.0,55.5,FIN,1996,Summer,Atlanta,Sailing,sailing,Windsurfer,NaN,Finland
26,13,F,34.0,159.0,55.5,FIN,2000,Summer,Sydney,Sailing,sailing,Windsurfer,NaN,Finland
29,18,M,31.0,189.0,130.0,FIN,2000,Summer,Sydney,Athletics,heavyset_field_events,Shot Put,NaN,Finland
50,22,F,22.0,170.0,125.0,ROU,2016,Summer,Rio de Janeiro,Weightlifting,weightlifting_super_heavyweight,Super-Heavyweight,NaN,Romania
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164980,135562,M,24.0,173.0,68.0,CZE,1972,Summer,Munich,Cycling,track_cycling,"Team Pursuit, 4,000 metres",NaN,Czech Republic
164981,135563,F,19.0,171.0,64.0,RUS,2000,Summer,Sydney,Athletics,track,4 x 400 metres Relay,Bronze,Russia
164982,135563,F,23.0,171.0,64.0,RUS,2004,Summer,Athina,Athletics,track,4 x 400 metres Relay,Silver,Russia
164984,135567,M,24.0,183.0,72.0,RUS,2000,Summer,Sydney,Rowing,rowing,Lightweight Coxless Fours,NaN,Russia


In [4]:
summer_df.medal = summer_df.medal.fillna('None')

/Users/alissa/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [5]:
summer_df

,id,sex,age,height,weight,noc,year,season,city,sport,event_category,event,medal,region
0,2,M,23.0,170.0,60.0,CHN,2012,Summer,London,Judo,judo_half_lightweight,Extra-Lightweight,None,China
25,13,F,30.0,159.0,55.5,FIN,1996,Summer,Atlanta,Sailing,sailing,Windsurfer,None,Finland
26,13,F,34.0,159.0,55.5,FIN,2000,Summer,Sydney,Sailing,sailing,Windsurfer,None,Finland
29,18,M,31.0,189.0,130.0,FIN,2000,Summer,Sydney,Athletics,heavyset_field_events,Shot Put,None,Finland
50,22,F,22.0,170.0,125.0,ROU,2016,Summer,Rio de Janeiro,Weightlifting,weightlifting_super_heavyweight,Super-Heavyweight,None,Romania
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164980,135562,M,24.0,173.0,68.0,CZE,1972,Summer,Munich,Cycling,track_cycling,"Team Pursuit, 4,000 metres",None,Czech Republic
164981,135563,F,19.0,171.0,64.0,RUS,2000,Summer,Sydney,Athletics,track,4 x 400 metres Relay,Bronze,Russia
164982,135563,F,23.0,171.0,64.0,RUS,2004,Summer,Athina,Athletics,track,4 x 400 metres Relay,Silver,Russia
164984,135567,M,24.0,183.0,72.0,RUS,2000,Summer,Sydney,Rowing,rowing,Lightweight Coxless Fours,None,Russia


In [6]:
cat_type = CategoricalDtype(categories=["Gold", "Silver", "Bronze", "None"], ordered=True)


In [7]:
summer_df["medals"] = summer_df['medal'].astype(cat_type)

/Users/alissa/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
summer_df = summer_df.dropna()

In [34]:
dmatri = dmatrices("medals ~ age + height + weight + C(event_category) + C(sex) + C(region)", summer_df, return_type="dataframe")

In [19]:
summer_df.medals.dtype

CategoricalDtype(categories=['Gold', 'Silver', 'Bronze', 'None'], ordered=True)

In [61]:
mod_prob = OrderedModel(summer_df['medals'],
                        matri,
                        distr='probit')

res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

         Current function value: 0.456599
         Iterations: 500
         Function evaluations: 505
         Gradient evaluations: 505


/Users/alissa/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                 medals   Log-Likelihood:                -53730.
Model:                   OrderedModel   AIC:                         1.080e+05
Method:            Maximum Likelihood   BIC:                         1.104e+05
Date:                Wed, 26 Jan 2022                                         
Time:                        11:58:38                                         
No. Observations:              117675                                         
Df Residuals:                  117425                                         
Df Model:                         250                                         
=========================================================================================================================
                                                            coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------
C(event_category)[T.boxing_flyweight]                    -0.6714      0.074     -9.069      0.000      -0.816      -0.526
C(event_category)[T.boxing_heavyweight]                  -0.4073      0.084     -4.835      0.000      -0.572      -0.242
C(event_category)[T.boxing_lightweight]                  -0.5122      0.064     -8.019      0.000      -0.637      -0.387
C(event_category)[T.boxing_middleweight]                 -0.3897      0.073     -5.359      0.000      -0.532      -0.247
C(event_category)[T.boxing_welterweight]                 -0.4034      0.065     -6.201      0.000      -0.531      -0.276
C(event_category)[T.diving]                              -0.1707      0.054     -3.143      0.002      -0.277      -0.064
C(event_category)[T.equestrianism]                       -0.1615      0.048     -3.359      0.001      -0.256      -0.067
C(event_category)[T.fencing]                             -0.1632      0.045     -3.637      0.000      -0.251      -0.075
C(event_category)[T.field_events]                         0.2228      0.050      4.461      0.000       0.125       0.321
C(event_category)[T.gymnastics]                           0.3026      0.045      6.770      0.000       0.215       0.390
C(event_category)[T.heavyset_field_events]                0.4079      0.054      7.524      0.000       0.302       0.514
C(event_category)[T.judo_half_heavyweight]               -0.1712      0.078     -2.192      0.028      -0.324      -0.018
C(event_category)[T.judo_half_lightweight]               -0.4423      0.068     -6.518      0.000      -0.575      -0.309
C(event_category)[T.judo_heavyweight]                     0.0097      0.089      0.110      0.913      -0.165       0.184
C(event_category)[T.judo_lightweight]                    -0.2990      0.065     -4.629      0.000      -0.426      -0.172
C(event_category)[T.judo_middleweight]                   -0.2232      0.080     -2.776      0.006      -0.381      -0.066
C(event_category)[T.road_cycling]                         0.2602      0.054      4.829      0.000       0.155       0.366
C(event_category)[T.rowing]                              -0.3754      0.044     -8.623      0.000      -0.461      -0.290
C(event_category)[T.sailing]                             -0.2556      0.054     -4.701      0.000      -0.362      -0.149
C(event_category)[T.shooting]                             0.2079      0.047      4.444      0.000       0.116       0.300
C(event_category)[T.slalom]                              -0.1988      0.068     -2.920      0.004      -0.332      -0.065
C(event_category)[T.straight_race]                       -0.2639      0.046     -5.745      0.000      -0.354      -0.174
C(event_category)[T.swimming]                            -0.0266      0.043     -0.624      0.532      -0.110   

In [57]:
dmatri[1]['Intercept'].unique()

array([1.])

In [22]:
summer_df.age

0         23.0
25        30.0
26        34.0
29        31.0
50        22.0
          ... 
164980    24.0
164981    19.0
164982    23.0
164984    24.0
164985    28.0
Name: age, Length: 117675, dtype: float64

In [21]:
summer_df.isnull().values.any()

False

In [42]:
dmatri[1]

,Intercept,C(event_category)[T.boxing_flyweight],C(event_category)[T.boxing_heavyweight],C(event_category)[T.boxing_lightweight],C(event_category)[T.boxing_middleweight],C(event_category)[T.boxing_welterweight],C(event_category)[T.diving],C(event_category)[T.equestrianism],C(event_category)[T.fencing],C(event_category)[T.field_events],...,C(region)[T.Venezuela],C(region)[T.Vietnam],"C(region)[T.Virgin Islands, British]","C(region)[T.Virgin Islands, US]",C(region)[T.Yemen],C(region)[T.Zambia],C(region)[T.Zimbabwe],age,height,weight
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,170.0,60.0
25,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,159.0,55.5
26,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,159.0,55.5
29,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,189.0,130.0
50,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,170.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164980,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,173.0,68.0
164981,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,171.0,64.0
164982,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,171.0,64.0
164984,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,183.0,72.0


In [56]:
sorted(summer_df.event_category.unique())

['archery',
 'boxing_flyweight',
 'boxing_heavyweight',
 'boxing_lightweight',
 'boxing_middleweight',
 'boxing_welterweight',
 'diving',
 'equestrianism',
 'fencing',
 'field_events',
 'gymnastics',
 'heavyset_field_events',
 'judo_half_heavyweight',
 'judo_half_lightweight',
 'judo_heavyweight',
 'judo_lightweight',
 'judo_middleweight',
 'road_cycling',
 'rowing',
 'sailing',
 'shooting',
 'slalom',
 'straight_race',
 'swimming',
 'table_tennis',
 'tennis',
 'track',
 'track_cycling',
 'weightlifting_featherweight',
 'weightlifting_heavy_middleweight',
 'weightlifting_heavyweight',
 'weightlifting_light_middleweight',
 'weightlifting_super_heavyweight',
 'wrestling_featherweight',
 'wrestling_flyweight',
 'wrestling_heavyweight',
 'wrestling_middleweight',
 'wrestling_super_heavyweight',
 'wrestling_welterweight']

In [59]:
matri = dmatri[1].drop('Intercept', axis = 1)

In [60]:
matri

,C(event_category)[T.boxing_flyweight],C(event_category)[T.boxing_heavyweight],C(event_category)[T.boxing_lightweight],C(event_category)[T.boxing_middleweight],C(event_category)[T.boxing_welterweight],C(event_category)[T.diving],C(event_category)[T.equestrianism],C(event_category)[T.fencing],C(event_category)[T.field_events],C(event_category)[T.gymnastics],...,C(region)[T.Venezuela],C(region)[T.Vietnam],"C(region)[T.Virgin Islands, British]","C(region)[T.Virgin Islands, US]",C(region)[T.Yemen],C(region)[T.Zambia],C(region)[T.Zimbabwe],age,height,weight
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,170.0,60.0
25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,159.0,55.5
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,159.0,55.5
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,189.0,130.0
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,170.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,173.0,68.0
164981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,171.0,64.0
164982,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,171.0,64.0
164984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,183.0,72.0


predicted = res_log.model.predict(res_log.params, exog=data_student[['pared', 'public', 'gpa']])
predicted

In [62]:
professional = {'age':[33, 22, 35, 26],
        'weight':[40, 84, 67, 63],
        'height':[159, 184, 183,163],
        'event_category':['track','swimming', 'speed_skating', 'archery'],
        'sex':['F', 'M', 'M', 'F'],
        'region':['Kenya', 'Singapore', 'Sweden', 'USA']}
professional_df = pd.DataFrame(professional)

In [77]:
matri[0]

KeyError: 0

In [66]:
res_prob.save('summer_fit.pickle')

In [67]:
res_prob.transform(professional_df)

AttributeError: 'OrderedResults' object has no attribute 'transform'

In [68]:
import pickle

In [72]:
import statsmodels.api as sm


In [73]:
new_results = sm.load('summer_fit.pickle')

In [78]:
new_results.predict(matri.iloc[:2])

,0,1,2,3
0,0.098083,0.084413,0.082113,0.735391
25,0.021397,0.029203,0.035941,0.913459


In [84]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [85]:
column_tran = make_column_transformer(
                (OneHotEncoder(sparse=False,handle_unknown='ignore'),["sex","event_category", "region"]),
                remainder = "passthrough")

In [86]:
pipe = make_pipeline(column_tran, mod_prob)

In [98]:
pipe.fit(olympic_data, summer_df['medals'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [88]:
olympic_data = summer_df[["sex","event_category","region","age","height","weight"]]

In [89]:
olympic_data.head()

,sex,event_category,region,age,height,weight
0,M,judo_half_lightweight,China,23.0,170.0,60.0
25,F,sailing,Finland,30.0,159.0,55.5
26,F,sailing,Finland,34.0,159.0,55.5
29,M,heavyset_field_events,Finland,31.0,189.0,130.0
50,F,weightlifting_super_heavyweight,Romania,22.0,170.0,125.0


In [91]:
olympic_medals = summer_df[['medals']]

In [92]:
olympic_medals

,medals
0,None
25,None
26,None
29,None
50,None
...,...
164980,None
164981,Bronze
164982,Silver
164984,None


In [97]:
matri

,C(event_category)[T.boxing_flyweight],C(event_category)[T.boxing_heavyweight],C(event_category)[T.boxing_lightweight],C(event_category)[T.boxing_middleweight],C(event_category)[T.boxing_welterweight],C(event_category)[T.diving],C(event_category)[T.equestrianism],C(event_category)[T.fencing],C(event_category)[T.field_events],C(event_category)[T.gymnastics],...,C(region)[T.Venezuela],C(region)[T.Vietnam],"C(region)[T.Virgin Islands, British]","C(region)[T.Virgin Islands, US]",C(region)[T.Yemen],C(region)[T.Zambia],C(region)[T.Zimbabwe],age,height,weight
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,170.0,60.0
25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,159.0,55.5
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,159.0,55.5
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,189.0,130.0
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,170.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,173.0,68.0
164981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,171.0,64.0
164982,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,171.0,64.0
164984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,183.0,72.0


In [99]:
X_summer = (summer_df.drop(["medals"], axis=1))
y_summer = np.array(summer_df["medals"])

In [103]:
Xw_train, Xw_test, yw_train, yw_test = train_test_split(X_summer, y_summer, test_size=0.2, random_state=42)

In [105]:
pipe.fit(Xw_train, yw_train)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()